In [11]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

In [12]:
client = weaviate.connect_to_local(
    port=8080,
    grpc_port=50051,
    additional_config=AdditionalConfig(
        timeout=Timeout(init=200, query=3000, insert=120) 
    ),
    skip_init_checks=True
)


In [13]:
import requests

file = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = file.json()
data

[{'Category': 'SCIENCE',
  'Question': 'This organ removes excess glucose from the blood & stores it as glycogen',
  'Answer': 'Liver'},
 {'Category': 'ANIMALS',
  'Question': "It's the only living mammal in the order Proboseidea",
  'Answer': 'Elephant'},
 {'Category': 'ANIMALS',
  'Question': 'The gavial looks very much like a crocodile except for this bodily feature',
  'Answer': 'the nose or snout'},
 {'Category': 'ANIMALS',
  'Question': 'Weighing around a ton, the eland is the largest species of this animal in Africa',
  'Answer': 'Antelope'},
 {'Category': 'ANIMALS',
  'Question': 'Heaviest of all poisonous snakes is this North American rattlesnake',
  'Answer': 'the diamondback rattler'},
 {'Category': 'SCIENCE',
  'Question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  'Answer': 'species'},
 {'Category': 'SCIENCE',
  'Question': 'A metal that is ductile can be pulled into this while cold & under pressur

In [14]:
import weaviate.classes as wvc

In [15]:
class_obj = {
    "class": "Question",
    "description": "A question",
    "vectorizer": "text2vec-ollama",
    "moduleConfig": {
      "text2vec-ollama": {
        "apiEndpoint": "http://ollama:11434",
        "model": "nomic-embed-text"
      },
      "generative-ollama": {
        "apiEndpoint": "http://ollama:11434",
        "model": "phi"
      }
    }
  }

In [16]:
client.collections.delete('Question')
questions = client.collections.create_from_dict(class_obj)

In [17]:
questions = client.collections.get("Question")
questions.data.insert_many(data)

BatchObjectReturn(all_responses=[UUID('5437eab7-be4d-4569-b487-22332b358e41'), UUID('43b44e65-135d-4585-9b0d-d7ba8bcfa9e9'), UUID('2d47afd8-3489-4769-820a-099b1de49499'), UUID('72a9a5a0-63ff-4cd4-9e35-93070654f680'), UUID('ffbdb9a8-a902-4475-a13a-707fc8200b7f'), UUID('eab0a753-7fef-4588-b929-bb509873aff8'), UUID('b3d517a9-597d-4788-8191-622ebadf5264'), UUID('b20c0ea6-fc50-4f22-a9f2-1020366a80fb'), UUID('bfc0d53e-12c8-4e82-a78e-15faf8fecc5b'), UUID('4ebe48db-3612-4e35-9634-b4202da4d6ff')], elapsed_seconds=33.66621398925781, errors={}, uuids={0: UUID('5437eab7-be4d-4569-b487-22332b358e41'), 1: UUID('43b44e65-135d-4585-9b0d-d7ba8bcfa9e9'), 2: UUID('2d47afd8-3489-4769-820a-099b1de49499'), 3: UUID('72a9a5a0-63ff-4cd4-9e35-93070654f680'), 4: UUID('ffbdb9a8-a902-4475-a13a-707fc8200b7f'), 5: UUID('eab0a753-7fef-4588-b929-bb509873aff8'), 6: UUID('b3d517a9-597d-4788-8191-622ebadf5264'), 7: UUID('b20c0ea6-fc50-4f22-a9f2-1020366a80fb'), 8: UUID('bfc0d53e-12c8-4e82-a78e-15faf8fecc5b'), 9: UUID('4eb

In [18]:
response = questions.query.near_text(
        query="biology",
        limit=2
    )

print(response.objects[0].properties)
print(response.objects[1].properties)

{'answer': 'Liver', 'question': 'This organ removes excess glucose from the blood & stores it as glycogen', 'category': 'SCIENCE'}
{'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}


In [19]:
response = questions.query.near_text(
        query="biology",
        limit=2,
        filters=wvc.query.Filter.by_property("category").equal("ANIMALS")
)

print(response.objects[0].properties)

{'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}


In [20]:
response = questions.generate.near_text(
        query="biology",
        limit=2,
        single_prompt="Explain {answer} as you might to a five-year-old."
    )

print(response.objects[0].generated)

 Sure, I can do that! Your liver is a very important organ in your body. It helps your body in many ways. One of its jobs is to filter out things from your blood, like toxins and other harmful substances. Another job is to make sure you have enough energy by storing fat for later use. The liver also helps your body absorb certain nutrients that you get from food. So as you can see, the liver plays a very important role in keeping your body healthy!



In [21]:
response = questions.generate.near_text(
        query="biology",
        limit=2,
        grouped_task="Write a tweet with emojis about these facts."
)

print(response.generated)

 🍎🍌🥕 Liver is an amazing organ that helps regulate blood sugar levels by storing excess glucose as glycogen! 🌤️🔄 Did you know changes in the troposphere's layers can influence our daily weather patterns? #LiverLove #ScienceFacts 🧪🔬

